In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from patsy import dmatrix
from time import time
from datetime import datetime as dt
plt.rc('font', family='SimHei', size=14)  # 中文显示必须
%matplotlib inline
no_uf = pd.read_csv(r'C:\Users\Administrator\Desktop\news_n.csv', encoding='utf-8')
yes_uf = pd.read_csv(r'C:\Users\Administrator\Desktop\news_y.csv', encoding='utf-8')

df = pd.concat([yes_uf, no_uf], ignore_index=True)  # ignore_index 重新生成index
df_text = df.applymap(lambda x: x.strip() if isinstance(x, str) else np.nan)
# df_text['text']
# df['wanted']

In [ ]:
import jieba
import jieba.posseg as pseg
import re
corpus = []
# pattern = r'^[0-9]*$'
pattern = '((?=[\x21-\x7e]+)[^A-Za-z])'
for t in df_text['text']:
    w = ''
    words = pseg.cut(str(t))
    for key in words:
        # 对key.word中的数字和特殊字符进行过滤.
        isNum = re.search(pattern, str(key.word))
        if not isNum:
            w += (key.word + ' ')
    corpus.append(w)
print(len(corpus))

In [ ]:
# stop_words = pd.read_csv(r'C:\Users\Administrator\Desktop\stop_word.csv',encoding = 'utf-8')
import os
import sys
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC

vectorizer = CountVectorizer()  # 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
transformer = TfidfTransformer()  # 该类会统计每个词语的tf-idf权值

tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus))  # 第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵

word = vectorizer.get_feature_names()  # 获取词袋模型中的所有词语
weight = tfidf.toarray()  # 将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重

# for i in range(len(weight)):  # 打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for便利某一类文本下的词语权重
#     print(u"######################################这里输出第", i,
#           u"类文本的词语tf-idf权重#############################################")
#     for j in range(len(word)):
#         print(word[j], '    ', weight[i][j])

In [44]:
# 将tf-idf矩阵传递给SVM模型
clf = LinearSVC()
model = clf.fit(tfidf, df['wanted'])
model

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [47]:
#　将新下载的文章转为DataFrame
test_uf = pd.read_csv(r'C:\Users\Administrator\Desktop\test.csv', encoding='utf-8')

# test_uf

# 将数据转为向量矩阵
# vec = CountVectorizer()  # 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
tran = TfidfTransformer()  # 该类会统计每个词语的tf-idf权值
# vectran = tran.fit_transform(vec.fit_transform(test_uf['text']))  # 第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵

test_matrix=tran.transform(test_uf['text'])
model.predict(test_matrix)

# results = pd.DataFrame(model.predict(test_matrix), columns = ['wanted'])

ValueError: could not convert string to float: '\u3000\u3000据最新消息显示，除了新款TLS相机外，徕卡今年可能还将会有一款新机发布。得出该结论的依据主要是徕卡今日刚刚在印度尼西亚通信局注册了一款代号为“7323”的新机，注册信息显示新机依旧为德国制造。目前，尚没有更多其他信息流出。此外，还有消息称徕卡将会在今年推出新款Noctilux M 75mm f/1.25 ASPH镜头。'